In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback, LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import preprocess_input

In [5]:
# **Step 1: Define data generator**
class DataGenerator(Sequence):
    def __init__(self, X_path, y_path, indices, batch_size):
        self.X = np.memmap(X_path, dtype='float32', mode='r', shape=(726, 15, 224, 224, 3))
        self.y = np.memmap(y_path, dtype='int32', mode='r', shape=(726,))
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[batch_indices]
        X_batch = preprocess_input(X_batch)  # Preprocess for ResNet
        y_batch = np.eye(2)[self.y[batch_indices]]  # One-hot encode
        return X_batch, y_batch
    
class StepTimerCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n--- Starting Epoch {epoch + 1} ---")
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        print(f"--- Epoch {epoch + 1} completed in {epoch_time:.2f} seconds ---\n")

    def on_train_batch_begin(self, batch, logs=None):
        self.step_start_time = time.time()
        print(f"Step {batch + 1}/{self.params['steps']} - ", end="")

    def on_train_batch_end(self, batch, logs=None):
        step_time = time.time() - self.step_start_time
        print(f"Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Time: {step_time:.2f} seconds")
        
class BatchEarlyStopping(Callback):
    def __init__(self, monitor='loss', threshold=0.1, patience=5):
        """
        Early stopping within the same epoch based on a monitored metric.
        Args:
            monitor: Metric to monitor ('loss', 'accuracy', etc.).
            threshold: Threshold value for stopping (e.g., loss < 0.1).
            patience: Number of batches to wait for improvement before stopping.
        """
        super().__init__()
        self.monitor = monitor
        self.threshold = threshold
        self.patience = patience
        self.wait = 0

    def on_train_batch_end(self, batch, logs=None):
        current_value = logs.get(self.monitor)
        if current_value is not None:
            # Check if the monitored metric meets the threshold
            if current_value < self.threshold:
                self.wait += 1
                if self.wait >= self.patience:
                    print(f"\nEarly stopping triggered at batch {batch + 1}: {self.monitor} = {current_value:.4f}")
                    self.model.stop_training = True
            else:
                self.wait = 0  # Reset patience if condition is not met

# Instantiate the batch-level early stopping callback
batch_early_stopping_callback = BatchEarlyStopping(
    monitor='loss',      # Metric to monitor
    threshold=0.1,       # Stop if loss goes below this value
    patience=2           # Number of consecutive batches meeting the condition
)

# **Step 2: Build the CNN-LSTM model**
def build_cnn_lstm_model(seq_length, height, width, channels, num_classes):
    # Use MobileNet as the base model
    base_model = ResNet(weights='imagenet', include_top=False, input_shape=(height, width, channels))
    base_model.trainable = False  # Freeze the pre-trained model

    # Input for the sequence of frames
    sequence_input = Input(shape=(seq_length, height, width, channels))
    
    # TimeDistributed wrapper to apply the CNN to each frame
    cnn_features = TimeDistributed(base_model)(sequence_input)
    flattened_features = TimeDistributed(Flatten())(cnn_features)

    # Single LSTM layer for temporal modeling
    lstm_out = LSTM(128)(flattened_features)
    lstm_out = Dropout(0.5)(lstm_out)

    # Single fully connected layer and output
    dense_out = Dense(64, activation='relu')(lstm_out)
    dense_out = Dropout(0.5)(dense_out)
    output = Dense(num_classes, activation='softmax')(dense_out)

    # Compile the model
    model = Model(inputs=sequence_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the model
cnn_lstm_model = build_cnn_lstm_model(seq_length=15, height=224, width=224, channels=3, num_classes=2)
cnn_lstm_model.summary()

# **Step 3: Split dataset into train and validation**
indices = np.arange(726)
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', train_indices, batch_size=11)
val_gen = DataGenerator(r'E:\PosePerfect\Dataset Creation\X_combined.dat', r'E:\PosePerfect\Dataset Creation\y_combined.dat', val_indices, batch_size=11)

# **Step 4: Set up checkpoint callback**
checkpoint_dir = './Checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_val_loss_{val_loss:.2f}.keras')
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',  # Metric to monitor
    save_best_only=False,  # Save at every epoch regardless of performance
    save_weights_only=False,  # Save the entire model (not just weights)
    verbose=1
)

# **Step 6: Learning Rate Scheduler**
def scheduler(epoch, lr):
    if epoch < 2:  # Keep the initial learning rate for the first 2 epochs
        return lr
    return lr * 0.9  # Decay the learning rate by 10% after every epoch

lr_callback = LearningRateScheduler(scheduler)

step_timer_callback = StepTimerCallback()

# Step 7: Train the model 
cnn_lstm_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback, lr_callback, step_timer_callback, batch_early_stopping_callback],
    verbose=1  # Ensure verbose is enabled
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 15, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 7, 7, 2048) │    23,587,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 15, 100352)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 15, 128)        │    51,446,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,085,538 (286.43 MB)

 Trainable params: 51,497,826 (196.45 MB)

 Non-trainable params: 23,587,712 (89.98 MB)


--- Starting Epoch 1 ---
Epoch 1/10
Step 1/52 - Loss: 0.6731, Accuracy: 0.6364, Time: 246.63 seconds
 1/52 ━━━━━━━━━━━━━━━━━━━━ 3:36:06 254s/step - accuracy: 0.6364 - loss: 0.6731Step 2/52 - Loss: 0.7258, Accuracy: 0.6364, Time: 17.37 seconds
 2/52 ━━━━━━━━━━━━━━━━━━━━ 14:44 18s/step - accuracy: 0.6364 - loss: 0.6995   Step 3/52 - Loss: 0.7438, Accuracy: 0.5455, Time: 16.23 seconds
 3/52 ━━━━━━━━━━━━━━━━━━━━ 13:58 17s/step - accuracy: 0.6061 - loss: 0.7142Step 4/52 - Loss: 0.7751, Accuracy: 0.5227, Time: 10.54 seconds
 4/52 ━━━━━━━━━━━━━━━━━━━━ 12:00 15s/step - accuracy: 0.5852 - loss: 0.7294Step 5/52 - Loss: 0.7706, Accuracy: 0.4727, Time: 14.33 seconds
 5/52 ━━━━━━━━━━━━━━━━━━━━ 11:38 15s/step - accuracy: 0.5627 - loss: 0.7377Step 6/52 - Loss: 0.7490, Accuracy: 0.4848, Time: 10.21 seconds
 6/52 ━━━━━━━━━━━━━━━━━━━━ 10:42 14s/step - accuracy: 0.5497 - loss: 0.7396Step 7/52 - Loss: 0.7470, Accuracy: 0.4805, Time: 9.65 seconds
 7/52 ━━━━━━━━━━━━━━━━━━━━ 9:58 13s/step - accuracy: 0.5399